# Subpopulations Meta-Learning Distributions

In [ ]:
import os
from os.path import join

import numpy as np
import pandas as pd

from common import MCLASSES, filter_msets, read_toml, save_toml

In [ ]:
def filter_subpop(df, filter_df, col, val):
    msets = ['mval', 'mtst']
    if isinstance(val, str):
        exclude = ~(df[col] == val)
        filter_df.loc[exclude, msets] = 0
    else:
        if len(val) == 1:
            (a_left, a_right), = val
            exclude = ~(df[col].between(a_left, a_right, inclusive='both'))
            filter_df.loc[exclude, msets] = 0
        else:
            (a_left, a_right), (b_left, b_right) = val
            exclude = ~(df[col].between(a_left, a_right, inclusive='both')|
                        df[col].between(b_left, b_right, inclusive='both'))
            filter_df.loc[exclude, msets] = 0
    return filter_df

In [ ]:
metachest_dir = read_toml('config.toml')['metachest_dir']
mtl_dir = join(metachest_dir, 'mtl')
os.makedirs(mtl_dir, exist_ok=True)

mclasses = MCLASSES
df = pd.read_csv(join(metachest_dir, 'metachest.csv'))

subpops = [
    # name     col     mval/mtst
    ['center', 'age',  [[31,62]]],
    ['tails',  'age',  [[10,30],[63,80]]],
    ['female', 'sex',  'f'],
    ['male',   'sex',  'm'],
    ['ap',     'view', 'ap'],
    ['pa',     'view', 'pa']
]

for subpop in subpops:

    name, col, val = subpop
    filter_df = pd.DataFrame(
        np.ones([df.shape[0], 3], dtype=int),
        columns=mclasses.keys()
    )

    filter_subpop(df, filter_df, col, val)
    filter_msets(df, filter_df, mclasses)

    path = join(mtl_dir, f'{col}_{name}')
    save_toml(f'{path}.toml', mclasses)
    filter_df.to_csv(f'{path}.csv', index=False)
